In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM,Dense ,Dropout,Bidirectional

In [ ]:
data = pd.read_csv(r"april_to_june.csv",sep=",")

In [ ]:
district = data[data.district == "Adilabad"]
district = district.drop(["dmcode","district","mandal","wind_speed_max","wind_speed_min"],axis=1)
district["temp"] = (district["temp_min"] + district["temp_max"])/2
district["humidity"] = (district["humidity_min"] + district["humidity_max"])/2
district = district.drop(["temp_min","temp_max","humidity_min","humidity_max"],axis = 1)

In [ ]:
district.head()

,date,rain,temp,humidity
0,2020-04-01,0.0,29.80,57.35
1,2020-04-02,0.0,30.15,55.40
2,2020-04-03,0.0,31.60,46.40
3,2020-04-04,1.3,31.05,62.30
4,2020-04-05,0.0,30.60,51.15


In [ ]:
district = district.reset_index(drop=True)
length = district.shape[0]
train_size = int(length*7/10)
training_set = district[:train_size]
training_set = training_set.drop(["date"],axis=1)

In [ ]:
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
training_set_scaled

array([[0.        , 0.39880952, 0.52407034],
       [0.        , 0.41964286, 0.50158547],
       [0.        , 0.50595238, 0.39780917],
       ...,
       [0.10672854, 0.125     , 0.88036898],
       [0.14269142, 0.1547619 , 0.83597579],
       [0.00812065, 0.14583333, 0.8526953 ]])

In [ ]:
x_train = []
y_train = []
n_future = 6 # next 4 days temperature forecast
n_past = 40 # Past 30 days 
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])     
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

In [ ]:
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc'])
regressor.fit(x_train, y_train, epochs=500,batch_size=32 )

Epoch 1/500
71/71 [==============================] - 4s 62ms/step - loss: 0.0158 - acc: 0.1584
Epoch 2/500
71/71 [==============================] - 4s 63ms/step - loss: 0.0150 - acc: 0.1867
Epoch 3/500
71/71 [==============================] - 4s 63ms/step - loss: 0.0147 - acc: 0.1615
Epoch 4/500
71/71 [==============================] - 5s 64ms/step - loss: 0.0146 - acc: 0.1765
Epoch 5/500
71/71 [==============================] - 5s 64ms/step - loss: 0.0145 - acc: 0.1726
Epoch 6/500
71/71 [==============================] - 5s 64ms/step - loss: 0.0144 - acc: 0.1611
Epoch 7/500
71/71 [==============================] - 5s 64ms/step - loss: 0.0143 - acc: 0.1743
Epoch 8/500
71/71 [==============================] - 5s 65ms/step - loss: 0.0144 - acc: 0.2097
Epoch 9/500
71/71 [==============================] - 5s 64ms/step - loss: 0.0143 - acc: 0.1633
Epoch 10/500
71/71 [==============================] - 5s 64ms/step - loss: 0.0143 - acc: 0.1934
Epoch 11/500
71/71 [=============================

In [ ]:
testdataset = district[train_size+1:]
testdataset = testdataset.drop(["date"],axis=1)
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

In [ ]:
predicted_temperature = regressor.predict(testing)
print(predicted_temperature)
#predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))
predicted = sc.inverse_transform(predicted_temperature)

[[0.02830289 0.03721946 0.05675417 0.05975984 0.05839346 0.03808976]
 [0.02178546 0.05162836 0.06009634 0.05412932 0.05187536 0.12696737]
 [0.02791984 0.03319394 0.05865123 0.06084972 0.05578621 0.04237311]]


In [ ]:
to_predict = pd.read_csv(r"july_to_sept.csv",sep=",")
predicted_list = []
dis = to_predict.district.unique()
for i in dis:
    district = data[data.district == i]
    district = district.drop(["dmcode","district","mandal","wind_speed_max","wind_speed_min"],axis=1)
    district["temp"] = (district["temp_min"] + district["temp_max"])/2
    district["humidity"] = (district["humidity_min"] + district["humidity_max"])/2
    district = district.drop(["temp_min","temp_max","humidity_min","humidity_max"],axis = 1)
    district = district.reset_index(drop=True)
    size = int(length*7/10)
    print(i,district.shape)
    testdataset = district[-40:]
    print(i,testdataset.shape)
    testdataset = testdataset.drop(["date"],axis=1)
    print(i,testdataset.shape)
    testing = sc.transform(testdataset)
    testing = np.array(testing)
    testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))
    predicted_temperature = regressor.predict(testing)
    
    #predicted_temperature = sc.inverse_transform(predicted_temperature)
    predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))
    predicted = sc.inverse_transform(predicted_temperature)
    predicted_list.append((i,predicted[0][0],predicted[0][1],predicted[0][2]))
    print(predicted)

Adilabad (3294, 4)
Adilabad (40, 4)
Adilabad (40, 3)
[[ 0.68315846 23.655895   16.499844  ]
 [ 3.744356   23.479536   25.286854  ]
 [ 1.3927945  23.77497    16.516207  ]
 [ 3.965298   23.79993    22.811487  ]
 [ 2.3627608  23.522583   16.39708   ]
 [ 4.631086   23.859846   13.989257  ]]
Kumuram Bheem (2745, 4)
Kumuram Bheem (40, 4)
Kumuram Bheem (40, 3)
[[ 0.7361548 24.031239  17.035828 ]
 [ 3.6107724 23.553925  32.150734 ]
 [ 2.2878335 24.012718  17.149565 ]
 [ 4.912265  24.053135  21.034851 ]
 [ 1.976992  23.541994  16.037872 ]
 [ 4.1978273 23.835129  13.999575 ]]
Mancherial (3294, 4)
Mancherial (40, 4)
Mancherial (40, 3)
[[ 1.056579  24.199738  17.199575 ]
 [ 3.7333002 23.618906  32.69194  ]
 [ 3.072333  24.353064  17.354576 ]
 [ 5.8042464 24.347557  24.216604 ]
 [ 1.7533089 23.612085  16.647343 ]
 [ 4.7435346 24.021172  16.356426 ]]
Nirmal (3477, 4)
Nirmal (40, 4)
Nirmal (40, 3)
[[ 1.0077344 24.142658  17.023916 ]
 [ 3.7956533 23.588068  32.28463  ]
 [ 3.5677953 24.547968  17.19608

In [ ]:
dataset = []
for i in range(len(predicted_list)):
  predicted_list[i] = [predicted_list[i][0],predicted_list[i][1],predicted_list[i][2],predicted_list[i][3]]

In [ ]:
dataset = pd.DataFrame(predicted_list,columns=["district","rain","temp","humidity"])

In [ ]:
dataset.to_csv("temp_rainfall.csv")